In [1]:
import mlflow
import mlflow.pyfunc

import pandas as pd
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [10]:
#load data from experiment tracking that is in production - nb X_data are already transformed
with open('../data/test_data/202304-usage-experiment2.pkl', 'rb') as f: 
    X_train, y_train, X_test,  y_test = pickle.load(f)

In [11]:
#load model
TRACKING_URL = "http://34.68.82.207:5000"
mlflow.set_tracking_uri(TRACKING_URL)


In [12]:

logged_model = "models:/randomforest-scaled/Production"

In [13]:
mlflow.pyfunc.get_model_dependencies(logged_model)

/Users/isabellevea/anaconda3/envs/py11/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


2024/07/24 15:36:42 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /var/folders/dd/nb8m5vwd1sz3_s49jh1plcfr0000gn/T/tmpau94iqqi/requirements.txt'.


'/var/folders/dd/nb8m5vwd1sz3_s49jh1plcfr0000gn/T/tmpau94iqqi/requirements.txt'

In [34]:
%pip install -r /var/folders/dd/nb8m5vwd1sz3_s49jh1plcfr0000gn/T/tmpkv8uin21/requirements.txt

  Obtaining dependency information for mlflow==2.14.3 from https://files.pythonhosted.org/packages/97/a7/76efb3bf8d326cf3e3a700297fe6e10efa125b0b66ada6f79da8b03daabf/mlflow-2.14.3-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpickle==3.0.0 from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Obtaining dependency information for numpy==1.26.4 from https://files.pythonhosted.org/packages/a7/94/ace0fdea5241a27d13543ee117cbc65868e82213fb31a8eb7fe9ff23f313/numpy-1.26.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
  Obtaining dependency information for packaging==24.1 from https://files.pythonhosted.org/packages/08/aa/cc0199a5f0ad350994d660967a8efb233fe0416e4639146c089643407ce6/packaging-24.1-py3-none-any.whl.metadata
  

In [14]:
loaded_model = mlflow.pyfunc.load_model(logged_model)
print(f"This is the loaded model:{loaded_model}")

/Users/isabellevea/anaconda3/envs/py11/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


This is the loaded model:mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.sklearn
  run_id: c7f92afc141649028f9851d957d1a313



In [16]:
# loading the scaler and encoder from folders (need to figure out how to retrieve from mlflow later)

with open('../experiment_tracking/models/encoder-experiment4.pkl', 'rb') as f: 
    encoder = pickle.load(f)

with open('../experiment_tracking/models/scaler-experiment4.pkl', 'rb') as f: 
    scaler = pickle.load(f)

In [18]:
encoder

OneHotEncoder(handle_unknown='ignore')

In [19]:
scaler

StandardScaler()

In [17]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.sklearn
  run_id: c7f92afc141649028f9851d957d1a313

In [8]:
predictions = loaded_model.predict(X_test)

In [9]:
mean_squared_error(y_test.values, predictions)

18.409726623840115

In [40]:
data_processed = pd.read_parquet("../data/processed/202304-usage.parquet")

In [41]:
data_processed.head()

,net_usage,station_name,hour,day_of_week
0,1.0,2112 W Peterson Ave,0.0,Sunday
1,-1.0,2112 W Peterson Ave,0.0,Thursday
2,-1.0,2112 W Peterson Ave,0.0,Tuesday
3,-1.0,2112 W Peterson Ave,0.0,Wednesday
4,-1.0,2112 W Peterson Ave,1.0,Sunday


In [46]:
data_processed['station_name'].unique().tolist()

['2112 W Peterson Ave',
 '410',
 '63rd St Beach',
 '900 W Harrison St',
 'Aberdeen St & Jackson Blvd',
 'Aberdeen St & Monroe St',
 'Aberdeen St & Randolph St',
 'Ada St & Washington Blvd',
 'Adler Planetarium',
 'Albany Ave & 16th St',
 'Albany Ave & 26th St',
 'Albany Ave & Belmont Ave',
 'Albany Ave & Bloomingdale Ave',
 'Albany Ave & Douglas Blvd',
 'Albany Ave & Montrose Ave',
 'Altgeld Gardens',
 'Archer (Damen) Ave & 37th St',
 'Archer Ave & 43rd St',
 'Artesian Ave & 55th St',
 'Artesian Ave & Hubbard St',
 'Ashland Ave & 13th St',
 'Ashland Ave & 50th St',
 'Ashland Ave & 63rd St',
 'Ashland Ave & 66th St',
 'Ashland Ave & 74th St',
 'Ashland Ave & 78th St',
 'Ashland Ave & Archer Ave',
 'Ashland Ave & Augusta Blvd',
 'Ashland Ave & Belle Plaine Ave',
 'Ashland Ave & Blackhawk St',
 'Ashland Ave & Chicago Ave',
 'Ashland Ave & Division St',
 'Ashland Ave & Garfield Blvd',
 'Ashland Ave & Grace St',
 'Ashland Ave & Grand Ave',
 'Ashland Ave & Lake St',
 'Ashland Ave & McDowell 

In [47]:
data_processed['hour'].unique().tolist()

[0.0,
 1.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 4.0,
 2.0,
 3.0,
 5.0,
 6.0,
 23.0]